In [12]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import re
import string

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('stopwords')
nltk.download('vader_lexicon')


from collections import Counter
import plotly.express as px

from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns


sns.set(style="darkgrid")

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91626\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\91626\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
df = pd.read_excel("volc_suspect.xlsx")
df.head()

,date,user_id,text,act_sentiment
0,2024-04-13,Finchinator,Volcarona is one of the strongest win conditio...,pos
1,2024-04-14,Shaymin,this is the quietest opening to a OU suspect t...,pos
2,2024-04-15,serotine,"Easy echo above, Volcarona is just unbelievabl...",pos
3,2024-04-15,coralreaper,BAN. Volcarona might not be super broken compa...,pos
4,2024-04-15,daddybuzzwole,plot twist of the century: i actually don't ha...,neg


In [14]:
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91626\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
import re
thoughts = df['text']
corpus=[]

for i in range(0,len(df)):
    review = re.sub('[^a-zA-Z]', ' ', str(thoughts[i]))
    review = review.lower()
    review = review.split()
    stop_words=set(stopwords.words('English'))
    more_words=['like','volcarona','volc','tera']
    stop_words.update(more_words)
    review = [lemma.lemmatize(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

corpus[0:2]

['one strongest win condition history modern overused rising prominence debut generation bw remaining one better set option across last thireteen year losing hidden power gaining heavy duty boot reshaped last generation terastallization perhaps biggest impact presence ever generation controversial quickban earlier generation reintroduced flagship metagame hope would stay afloat time around however infamous quiver dancer frequently dubbed match moth controversial presence metagame',
 'quietest opening ou suspect test thread ever seen lmao anyways ban guy annoying shit fiery dance trolliest move ever created exist rng mu fish cheese mon meta gen already wide threat list giving option stupid deal way reliably beat volcs set unless u blissey clod troll use besides hard stall need defensive utility v kyurem kyurem check bulky set take spec draco u cannot seriously think counter lol look definition plz defensive utility flame body also coincedentally b rng nonsense end mon arsenal cheese eve

In [17]:
words_list = [word for line in corpus for word in line.split()]


from collections import Counter
word_counts = Counter(words_list).most_common(50)
words_df = pd.DataFrame(word_counts)
words_df.columns = ['word', 'frequency']
words_df.head()

,word,frequency
0,team,311
1,mon,297
2,check,287
3,set,271
4,broken,258


In [18]:
df['text'] = corpus
df.head()

,date,user_id,text,act_sentiment
0,2024-04-13,Finchinator,one strongest win condition history modern ove...,pos
1,2024-04-14,Shaymin,quietest opening ou suspect test thread ever s...,pos
2,2024-04-15,serotine,easy echo unbelievably hard account team build...,pos
3,2024-04-15,coralreaper,ban might super broken compared lot maniac use...,pos
4,2024-04-15,daddybuzzwole,plot twist century actually strong feeling tow...,neg


In [19]:
## text vectorisers ##


## bag or words ##
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()


## tf idf ##
from sklearn.feature_extraction.text import TfidfVectorizer
tif = TfidfVectorizer()
X1 = tif.fit_transform(corpus).toarray()


In [20]:
from sklearn.model_selection import train_test_split
y=pd.get_dummies(df['act_sentiment'])
y=y.iloc[:,1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 41)
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size = 0.20, random_state = 41)

In [21]:
# prediction models #


# PSA (Polarity Score Analysis) #
sid = SentimentIntensityAnalyzer()
ps = lambda x : sid.polarity_scores(x)
sentiment_scores = df.text.apply(ps)
sentiment_df = pd.DataFrame(data = list(sentiment_scores))
labelize = lambda x : 'neu' if x==0 else('pos' if x>0 else 'neg')
sentiment_df['label_psa'] = sentiment_df.compound.apply(labelize)


# VSA (Vader Sentiment Analyser) #
sid = SentimentIntensityAnalyzer()
def sentiment_Vader(text):
    over_all_polarity = sid.polarity_scores(text)
    if over_all_polarity['compound'] >= 0.05:
        return "pos"
    elif over_all_polarity['compound'] <= -0.05:
        return "neg"
    else:
        return "neutral"
vad = pd.DataFrame(data = list(df['text'].apply(lambda x: sentiment_Vader(x))))



# naive bayes #
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(X_train, y_train)
nb_model_1 = MultinomialNB().fit(X1_train, y_train)


# svm #
from sklearn import svm
clf = svm.SVC(kernel='rbf').fit(X_train, y_train)
clf_1 = svm.SVC(kernel='rbf').fit(X1_train, y_train)


# log regr #
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C = 10, solver = 'liblinear').fit(X_train,y_train)
LR_1 = LogisticRegression(C = 10, solver = 'liblinear').fit(X1_train,y_train)


In [22]:
# predictions and outputs for train/test set #

yhat_nb = nb_model.predict(X_test)
yhat_nb_1 = nb_model_1.predict(X1_test)

yhat_clf = clf.predict(X_test)
yhat_clf_1 = clf_1.predict(X1_test)

yhat_LR = LR.predict(X_test)
yhat_LR_1 = LR_1.predict(X1_test)

# predictions and outputs for whole dataset #

yhat_nb_full = nb_model.predict(X)
yhat_nb_1_full = nb_model_1.predict(X1)

yhat_clf_full = clf.predict(X)
yhat_clf_1_full = clf_1.predict(X1)

yhat_LR_full = LR.predict(X)
yhat_LR_1_full = LR_1.predict(X1)



df['PSA_label'] = sentiment_df['label_psa']
df['VSA_label'] = vad


df['VSA_label'][48] = 'neg'
df['PSA_label'][48] = 'neg'


df['BoW_Nb_label'] = yhat_nb_full
df['BoW_SVM_label'] = yhat_clf_full
df['BoW_LR_label'] = yhat_LR_full
df['TfIdf_Nb_label'] = yhat_nb_1_full
df['TfIdf_SVM_label'] = yhat_clf_1_full
df['Tfidf_LR_label'] = yhat_LR_1_full

df[0:7]


,date,user_id,text,act_sentiment,PSA_label,VSA_label,BoW_Nb_label,BoW_SVM_label,BoW_LR_label,TfIdf_Nb_label,TfIdf_SVM_label,Tfidf_LR_label
0,2024-04-13,Finchinator,one strongest win condition history modern ove...,pos,pos,pos,True,True,True,True,True,True
1,2024-04-14,Shaymin,quietest opening ou suspect test thread ever s...,pos,neg,neg,True,True,True,True,True,True
2,2024-04-15,serotine,easy echo unbelievably hard account team build...,pos,pos,pos,True,True,True,True,True,True
3,2024-04-15,coralreaper,ban might super broken compared lot maniac use...,pos,neg,neg,True,True,True,True,True,True
4,2024-04-15,daddybuzzwole,plot twist century actually strong feeling tow...,neg,neg,neg,False,False,False,False,False,False
5,2024-04-15,earthflax,people complaining reason ever since released ...,pos,neg,neg,True,True,True,True,True,True
6,2024-04-15,gummyrowlets,analysing obviously broken something flutter m...,pos,neg,neg,True,True,True,True,True,True


In [23]:
from sklearn.metrics import classification_report
print('---------------------------------------------------------------------------')
print("scores for BoW , Nb model : \n")
print(classification_report(y_test, yhat_nb))
print('---------------------------------------------------------------------------')
print("scores for BoW , SVM model : \n")
print(classification_report(y_test, yhat_clf))
print('---------------------------------------------------------------------------')
print("scores for BoW , LR model : \n")
print(classification_report(y_test, yhat_LR))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , Nb model : \n")
print(classification_report(y_test, yhat_nb_1))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , SVM model : \n")
print(classification_report(y_test, yhat_clf_1))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , LR model : \n")
print(classification_report(y_test, yhat_LR_1))
print('---------------------------------------------------------------------------')


---------------------------------------------------------------------------
scores for BoW , Nb model : 

              precision    recall  f1-score   support

       False       0.46      0.60      0.52        10
        True       0.73      0.61      0.67        18

    accuracy                           0.61        28
   macro avg       0.60      0.61      0.59        28
weighted avg       0.64      0.61      0.61        28

---------------------------------------------------------------------------
scores for BoW , SVM model : 

              precision    recall  f1-score   support

       False       1.00      0.10      0.18        10
        True       0.67      1.00      0.80        18

    accuracy                           0.68        28
   macro avg       0.83      0.55      0.49        28
weighted avg       0.79      0.68      0.58        28

---------------------------------------------------------------------------
scores for BoW , LR model : 

              precision    

In [24]:
## jaccard scores ##
from sklearn.metrics import jaccard_score
print("similarity score for BoW, Naive Bayes model is : ",jaccard_score(y_test, yhat_nb,pos_label=1))
print("similarity score for BoW, SVM model is : ",jaccard_score(y_test, yhat_clf,pos_label=1))
print("similarity score for BoW, LR model is : ",jaccard_score(y_test, yhat_LR,pos_label=1))
print("similarity score for TfIdf, Naive Bayes model is : ",jaccard_score(y_test, yhat_nb_1,pos_label=1))
print("similarity score for TfIdf, SVM model is : ",jaccard_score(y_test, yhat_clf_1,pos_label=1))
print("similarity score for TfIdf, LR model is : ",jaccard_score(y_test, yhat_LR_1,pos_label=1))

similarity score for BoW, Naive Bayes model is :  0.5
similarity score for BoW, SVM model is :  0.6666666666666666
similarity score for BoW, LR model is :  0.6363636363636364
similarity score for TfIdf, Naive Bayes model is :  0.6071428571428571
similarity score for TfIdf, SVM model is :  0.6428571428571429
similarity score for TfIdf, LR model is :  0.6


In [25]:
print("Metrics for a full dataset prediction : \n")

print('---------------------------------------------------------------------------')
print("scores for PSA model : \n")
print(classification_report(df['act_sentiment'], df['PSA_label']))
print("\nSimilarity score : ", jaccard_score(df['act_sentiment'], df['PSA_label'], pos_label='pos'))
print('---------------------------------------------------------------------------')
print("scores for VSA model : \n")
print(classification_report(df['act_sentiment'], df['VSA_label']))
print("\nSimilarity score : ", jaccard_score(df['act_sentiment'], df['VSA_label'], pos_label='pos'))
print('---------------------------------------------------------------------------')
print("scores for BoW , Nb model : \n")
print(classification_report(y, yhat_nb_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_nb_full))
print('---------------------------------------------------------------------------')
print("scores for BoW , SVM model : \n")
print(classification_report(y, yhat_clf_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_clf_full))
print('---------------------------------------------------------------------------')
print("scores for BoW , LR model : \n")
print(classification_report(y, yhat_LR_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_LR_full))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , Nb model : \n")
print(classification_report(y, yhat_nb_1_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_nb_1_full))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , SVM model : \n")
print(classification_report(y, yhat_clf_1_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_clf_1_full))
print('---------------------------------------------------------------------------')
print("scores for TfIdf , LR model : \n")
print(classification_report(y, yhat_LR_1_full))
print("\nSimilarity score : ", jaccard_score(y, yhat_LR_1_full))
print('---------------------------------------------------------------------------')

Metrics for a full dataset prediction : 

---------------------------------------------------------------------------
scores for PSA model : 

              precision    recall  f1-score   support

         neg       0.40      0.63      0.49        57
         pos       0.54      0.32      0.40        79

    accuracy                           0.45       136
   macro avg       0.47      0.47      0.44       136
weighted avg       0.48      0.45      0.44       136


Similarity score :  0.25
---------------------------------------------------------------------------
scores for VSA model : 

              precision    recall  f1-score   support

         neg       0.40      0.63      0.49        57
         pos       0.54      0.32      0.40        79

    accuracy                           0.45       136
   macro avg       0.47      0.47      0.44       136
weighted avg       0.48      0.45      0.44       136


Similarity score :  0.25
--------------------------------------------------

In [26]:
count_psa = 0
for i in range(len(df['PSA_label'])):
    if df['PSA_label'][i]=='pos':
        count_psa = count_psa + 1
psa=0
psa = count_psa
psa_score = (psa / len(df['PSA_label']) ) * 10
print("PSA sentiment score (out of 10) for the suspect test is : ", psa_score)




count_vsa = 0
for i in range(len(df['VSA_label'])):
    if df['VSA_label'][i]=='pos':
        count_vsa = count_vsa + 1
v=0
v = count_vsa
vsa_score = (v / len(df['VSA_label']) ) * 10
print("VSA sentiment score (out of 10) for the suspect test is : ", vsa_score)


count_bl = 0
for i in range(len(df['BoW_LR_label'])):
    if df['BoW_LR_label'][i]==1:
        count_bl = count_bl + 1
bl=0
bl = count_bl
bl_score = (bl / len(df['BoW_LR_label']) ) * 10
print("BL sentiment score (out of 10) for the suspect test is : ", bl_score)




PSA sentiment score (out of 10) for the suspect test is :  3.382352941176471
VSA sentiment score (out of 10) for the suspect test is :  3.382352941176471
BL sentiment score (out of 10) for the suspect test is :  5.808823529411765


In [27]:
count = 0
for i in range(len(df['act_sentiment'])):
    if df['act_sentiment'][i]=='pos':
        count = count + 1
p=0
p = count

score = (p / len(df['act_sentiment']) ) * 10
print("Actual sentiment score (out of 10) for the suspect test is : ", score)

Actual sentiment score (out of 10) for the suspect test is :  5.808823529411765


In [28]:
print("AND THE WINNER ISSSSS : BAG OF WORDS WITH LOGISTIC REGRESSION ")
print("TESTING ACCURACY = 71%")
print("SAMPLE ACCURACY = 94%")
print("ANDDDD THE FINAL SUSPECT SCORE BEINGGGG : 5.8/10 ")


AND THE WINNER ISSSSS : BAG OF WORDS WITH LOGISTIC REGRESSION 
TESTING ACCURACY = 71%
SAMPLE ACCURACY = 94%
ANDDDD THE FINAL SUSPECT SCORE BEINGGGG : 5.8/10 


In [29]:
print("the actual ban percentage was like 76% majority")
print("what the fuck did i do all this for")

the actual ban percentage was like 76% majority
what the fuck did i do all this for
